In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import yfinance as yf
import warnings
warnings.filterwarnings('ignore')

# Liquidity Adjusted VaR

Bid-ask spread is considered to be a proxy for modeling the liquidity in the market. Liquidity is a widespread financial phenomenon. Liquidity can be defined as the easiness of the trasaction with which assets can be sold in a very short time period without a significant impact on market price. 


In [2]:
bid_ask = pd.read_csv('bid_ask.csv')

In [3]:
bid_ask.head()

,Unnamed: 0,Date,EXCHCD,TICKER,COMNAM,BIDLO,ASKHI,PRC,VOL,RET,SHROUT,OPENPRC,vwretx
0,1031570,2019-01-02,3.0,INTC,INTEL CORP,45.77,47.470,47.08,18761673.0,0.003196,4564000.0,45.960,0.001783
1,1031571,2019-01-03,3.0,INTC,INTEL CORP,44.39,46.280,44.49,32254097.0,-0.055013,4564000.0,46.150,-0.021219
2,1031572,2019-01-04,3.0,INTC,INTEL CORP,45.54,47.570,47.22,35419836.0,0.061362,4564000.0,45.835,0.033399
3,1031573,2019-01-07,3.0,INTC,INTEL CORP,46.75,47.995,47.44,22724997.0,0.004659,4564000.0,47.100,0.009191
4,1031574,2019-01-08,3.0,INTC,INTEL CORP,46.78,48.030,47.74,22721240.0,0.006324,4564000.0,47.800,0.010240


The explanations on the most important columns are:
* BIDLO: Lowest bid price
* ASKHI: Highest Ask price
* VOL: Volume
* RET: Return
* SHROUT: Share Outstanding
* OPENPRC: Open Price
* vwretx: Volume weighted return

Following are the common bid-ask spreads used to model liquidity:
    
* Effective Spread
* Proportional Quoted Spread
* Quoted Spread
* Proportional Effective Spread
* Effective Cost 

\begin{equation}
\text{Effective Spread}=2|(P_t-P_{mid})|
\end{equation}




where $P_t$ is the price of trade at time t and $P_{mid}$ is the midpoint of the bid-ask offer ($(P_{ask}-P_{bid})/2$) prevailing at the time of the $t$.



\begin{equation}
\text{Proportional Quoted Spread}=(P_{ask} - P_{bid}) / P_{mid}
\end{equation}

where $P_{ask}$ is the ask price and $P_{bid}$ and $P_{mid}$ are bid price and mid price, respectively.



\begin{equation}
\text{Quoted Spread}= P_{ask} - P_{bid}
\end{equation}

\begin{equation}
\text{Proportional Effective Spread}= 2(|P_t-P_{mid}|) / P_{mid}
\end{equation}


\begin{equation}
\text{Effective Cost}= \left  \{
  \begin{array}{lr}
  (P_t-P_{mid})/P_{mid}\text{ for buyer initiated}\\
  (P_{mid}/P_t)/P_{mid}\text{ for seller initiated}\\
   \end{array}
\right.
\end{equation}


Buyer-initiated trade occurs when a trade is executed at a price abover quoted mid price. Similarly, seller-initiated trade occurs when a trade is executed at a price below than the quoted mid price.



In [4]:
bid_ask['mid_price'] = (bid_ask['ASKHI'] + bid_ask['BIDLO']) / 2
buyer_seller_initiated = []
for i in range(len(bid_ask)):
    if bid_ask['PRC'][i] > bid_ask['mid_price'][i]:
        buyer_seller_initiated.append(1)
    else:
        buyer_seller_initiated.append(0)
        
bid_ask['buyer_seller_init'] = buyer_seller_initiated   

In [5]:
effective_cost = []
for i in range(len(bid_ask)):
    if bid_ask['buyer_seller_init'][i] == 1:
        effective_cost.append((bid_ask['PRC'][i] - 
                               bid_ask['mid_price'][i]) / 
                               bid_ask['mid_price'][i])
    else:
        effective_cost.append((bid_ask['mid_price'][i] - 
                               bid_ask['PRC'][i])/
                               bid_ask['mid_price'][i])
bid_ask['effective_cost'] = effective_cost

In [6]:
bid_ask['quoted'] = bid_ask['ASKHI'] - bid_ask['BIDLO']
bid_ask['prop_quoted'] = (bid_ask['ASKHI'] - bid_ask['BIDLO']) /\
                         bid_ask['mid_price']
bid_ask['effective'] = 2 * abs(bid_ask['PRC'] - bid_ask['mid_price'])
bid_ask['prop_effective'] = 2 * abs(bid_ask['PRC'] - 
                                    bid_ask['mid_price']) /\
                                    bid_ask['PRC']

The last five columns of the `bid_ask` data contain our bid-ask spread calculations:

In [7]:
spread_meas = bid_ask.iloc[:, -5:]
spread_meas

,effective_cost,quoted,prop_quoted,effective,prop_effective
0,0.009867,1.70000,0.036465,0.92000,0.019541
1,0.018639,1.89000,0.041690,1.69000,0.037986
2,0.014284,2.03000,0.043604,1.33000,0.028166
3,0.001425,1.24500,0.026281,0.13500,0.002846
4,0.007067,1.25000,0.026369,0.67000,0.014034
...,...,...,...,...,...
751,0.000999,1.01000,0.007475,0.27000,0.002000
752,0.000502,0.65551,0.004856,0.13549,0.001004
753,0.000296,0.88000,0.006504,0.08000,0.000591
754,0.008140,2.80000,0.020911,2.18000,0.016414


Let us check the correlation structure of spread measures. The correlation table shows that effective cost and proportional effective spread are highly correlated. Whereas quoted and effective cost spread shows low correlation.

In [8]:
spread_meas.corr()

,effective_cost,quoted,prop_quoted,effective,prop_effective
effective_cost,1.000000,0.441290,0.727917,0.800894,0.999847
quoted,0.441290,1.000000,0.628526,0.717246,0.442053
prop_quoted,0.727917,0.628526,1.000000,0.514979,0.728687
effective,0.800894,0.717246,0.514979,1.000000,0.800713
prop_effective,0.999847,0.442053,0.728687,0.800713,1.000000
